In [ ]:
!pip install --upgrade --no-cache-dir gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install -U deepctr-torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Data Loader

In [ ]:
!gdown --id 1wg5oA66QtwEHqYUHfVzXsY1gtgqs1wJq

/usr/local/lib/python3.9/dist-packages/gdown/cli.py:126: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (uriginal): https://drive.google.com/uc?id=1wg5oA66QtwEHqYUHfVzXsY1gtgqs1wJq
From (redirected): https://drive.google.com/uc?id=1wg5oA66QtwEHqYUHfVzXsY1gtgqs1wJq&confirm=t&uuid=cd82033e-54a9-490b-bbad-1e5cd2a3da09
To: /content/criteo.zip
100% 87.9M/87.9M [00:00<00:00, 239MB/s]


In [ ]:
!unzip -o "criteo.zip"  -d  "/content"

Archive:  criteo.zip
  inflating: /content/train_1m.txt   


In [ ]:
label='Label'
target = [label]

In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


In [ ]:

continue_var = ['I' + str(i) for i in range(1, 14)]
cat_features = ['C' + str(i) for i in range(1,27)]
col_names_train = ['Label'] + continue_var + cat_features
col_names_test = col_names_train[1:]
train = pd.read_csv('train_1m.txt', sep='\t', 
                     names=col_names_train)

# train = pd.DataFrame()
# for i, chunk in enumerate(reader): 
   
#     train = pd.concat([train, chunk.sample(frac=1, replace=False, random_state=911)], axis=0)  
# train.head(1)

In [ ]:
train.shape

(1000000, 40)

In [ ]:
fill_mean = lambda x: x.fillna(x.mean())

for col in continue_var:
    train[col] = train[col].fillna(train[col].mean())
    train[col] = train[col].astype('float64')
train = train.fillna('-1', )


In [ ]:
from sklearn.preprocessing import LabelEncoder,MinMaxScaler
from sklearn.preprocessing import OneHotEncoder

for c in cat_features:
    le = LabelEncoder()
    le.fit(train[c])
    train[c] = le.transform(train[c])
# for i in cat_features:
#   data = pd.get_dummies(data, prefix=[i], columns = [i])

# ## EDA

classes=list(train[label].unique())
classes

classes = pd.value_counts(train[label], sort = True)


In [ ]:
# for c in cat_features:
#   if(len(list(train[c].unique()))<=20):
#    train = pd.get_dummies(train, prefix=[c], columns = [c])


In [ ]:
# cat_features=[]
# for i in list(train.columns):
#   if i[0]=='C':
#     cat_features.append(i)


In [ ]:
# mms = MinMaxScaler(feature_range=(0, 1))
# train[continue_var] = mms.fit_transform(train[continue_var])

## Train Data preprocessing

In [ ]:
import math
from sklearn.preprocessing import LabelEncoder ,MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.feature_selection import RFE
from sklearn.model_selection import GridSearchCV


In [ ]:
from deepctr_torch.inputs import SparseFeat, get_feature_names,DenseFeat
from deepctr_torch.models import DeepFM
import torch

In [ ]:
fixlen_feature_columns = [SparseFeat(feat, vocabulary_size=train[feat].max() + 1, embedding_dim=15)
                              for feat in cat_features] + [DenseFeat(feat, 1, )
                                                              for feat in continue_var]

In [ ]:
dnn_feature_columns = fixlen_feature_columns
linear_feature_columns = fixlen_feature_columns

feature_names = get_feature_names(
        linear_feature_columns + dnn_feature_columns)
feature_names

['C1',
 'C2',
 'C3',
 'C4',
 'C5',
 'C6',
 'C7',
 'C8',
 'C9',
 'C10',
 'C11',
 'C12',
 'C13',
 'C14',
 'C15',
 'C16',
 'C17',
 'C18',
 'C19',
 'C20',
 'C21',
 'C22',
 'C23',
 'C24',
 'C25',
 'C26',
 'I1',
 'I2',
 'I3',
 'I4',
 'I5',
 'I6',
 'I7',
 'I8',
 'I9',
 'I10',
 'I11',
 'I12',
 'I13']

In [ ]:

train, test = train_test_split(train, test_size=0.2)
train_model_input = {name: train[name] for name in feature_names}
test_model_input = {name: test[name] for name in feature_names}
train

,Label,I1,I2,I3,I4,I5,I6,I7,I8,I9,...,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26
884610,0,0.0000,2.0,12.000000,12.00000,5319.0,203.000000,9.0,41.0,285.0,...,9,3631,0,0,32570,0,2,25530,0,0
114145,0,1.0000,-1.0,21.094269,6.95427,1140.0,5.000000,1.0,7.0,11.0,...,9,2413,0,0,12153,10,4,34674,0,0
350124,0,0.0000,0.0,25.000000,2.00000,10529.0,268.000000,70.0,5.0,202.0,...,0,2145,227,2,192389,0,4,26534,11,1363
899978,0,15.0000,13.0,4.000000,4.00000,1012.0,8.000000,15.0,8.0,8.0,...,9,3723,0,0,212918,10,2,3766,0,0
303133,1,1.0000,31.0,21.094269,12.00000,1.0,1.000000,10.0,12.0,124.0,...,6,3539,0,0,217489,12,5,23031,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
784218,1,3.2384,1.0,2.000000,10.00000,2842.0,428.000000,12.0,45.0,310.0,...,9,3424,0,0,74315,10,4,35789,0,0
64284,1,13.0000,27.0,14.000000,10.00000,10.0,10.000000,18.0,8.0,32.0,...,9,2071,742,2,66244,0,10,25693,41,9121
281520,0,1.0000,300.0,21.094269,1.00000,616.0,1.000000,1.0,1.0,1.0,...,8,615,0,0,107528,0,2,28798,0,0
639713,0,0.0000,0.0,17.000000,8.00000,3470.0,29.000000,11.0,8.0,26.0,...,9,1387,227,2,136805,0,2,3766,58,16828


In [ ]:

device = 'cpu'
use_cuda = True
if use_cuda and torch.cuda.is_available():
        print('cuda ready...')
        device = 'cuda:0'

cuda ready...


## DEEPFM

In [ ]:

model = DeepFM(linear_feature_columns=linear_feature_columns, dnn_feature_columns=dnn_feature_columns,
                   task='binary',dnn_hidden_units=(512,128, 512,256),use_fm=True,  l2_reg_linear=0.1,
                   l2_reg_embedding=1e-1,  dnn_use_bn=True,device=device,dnn_activation='sigmoid')
model.compile("adagrad", "binary_crossentropy",
                  metrics=["binary_crossentropy", "auc"], )
history = model.fit(train_model_input, train[target].values, batch_size=2024, epochs=20, verbose=2,
                        validation_split=0.15)

cuda:0
Train on 680000 samples, validate on 120000 samples, 336 steps per epoch
Epoch 1/20
17s - loss:  3.1998 - binary_crossentropy:  2.0955 - auc:  0.6750 - val_binary_crossentropy:  0.6618 - val_auc:  0.7142
Epoch 2/20
17s - loss:  1.9809 - binary_crossentropy:  1.4917 - auc:  0.7065 - val_binary_crossentropy:  1.9272 - val_auc:  0.6828
Epoch 3/20
16s - loss:  1.6031 - binary_crossentropy:  1.2705 - auc:  0.7206 - val_binary_crossentropy:  0.8492 - val_auc:  0.7217
Epoch 4/20
17s - loss:  1.2662 - binary_crossentropy:  1.0415 - auc:  0.7361 - val_binary_crossentropy:  1.3934 - val_auc:  0.6986
Epoch 5/20
17s - loss:  1.1834 - binary_crossentropy:  0.9821 - auc:  0.7428 - val_binary_crossentropy:  1.2505 - val_auc:  0.7048
Epoch 6/20
17s - loss:  1.0664 - binary_crossentropy:  0.8996 - auc:  0.7492 - val_binary_crossentropy:  1.6575 - val_auc:  0.6923
Epoch 7/20
17s - loss:  0.9733 - binary_crossentropy:  0.8294 - auc:  0.7572 - val_binary_crossentropy:  0.9239 - val_auc:  0.7194
Epo

In [ ]:
from sklearn.metrics import log_loss, roc_auc_score,accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score,roc_curve
import statsmodels.api as sm
from sklearn.metrics import precision_score,recall_score
pred_ans = model.predict(test_model_input, 256)
ypred=[]
for i in pred_ans:
    ypred.append(round(i[0]))
print("")
print("test LogLoss", round(log_loss(test[target].values, pred_ans), 4))
print("test roc_auc_score", round(roc_auc_score(test[target].values, pred_ans), 4))
print("test f1 score", round(f1_score(test[target].values,ypred), 4))
print("test precision score", round(precision_score(test[target].values,ypred), 4))
print("test recall score", round(recall_score(test[target].values,ypred), 4))
print("test accuracy score", round(accuracy_score(test[target].values,ypred), 4))


test LogLoss 0.5897
test roc_auc_score 0.7482
test f1 score 0.4105
test precision score 0.619
test recall score 0.307
test accuracy score 0.7755


## xDeepFM

In [ ]:
from deepctr_torch.models import xDeepFM
model =  xDeepFM(linear_feature_columns=linear_feature_columns, dnn_feature_columns=dnn_feature_columns,
                   task='binary',dnn_hidden_units=(512,128, 512,256),  l2_reg_linear=0.1,
                   l2_reg_embedding=1e-1,  dnn_use_bn=True,device=device,dnn_activation='sigmoid')
model.compile("adagrad", "binary_crossentropy",
                  metrics=["binary_crossentropy", "auc"], )
history = model.fit(train_model_input, train[target].values, batch_size=2024, epochs=20, verbose=2,
                        validation_split=0.15)

cuda:0
Train on 680000 samples, validate on 120000 samples, 336 steps per epoch
Epoch 1/20
44s - loss:  3.3359 - binary_crossentropy:  2.1565 - auc:  0.6705 - val_binary_crossentropy:  1.0182 - val_auc:  0.7154
Epoch 2/20
41s - loss:  2.0363 - binary_crossentropy:  1.5257 - auc:  0.7037 - val_binary_crossentropy:  0.7714 - val_auc:  0.7270
Epoch 3/20
40s - loss:  1.5084 - binary_crossentropy:  1.1959 - auc:  0.7301 - val_binary_crossentropy:  1.7293 - val_auc:  0.6896
Epoch 4/20
40s - loss:  1.2903 - binary_crossentropy:  1.0344 - auc:  0.7646 - val_binary_crossentropy:  0.5560 - val_auc:  0.7306
Epoch 5/20
41s - loss:  0.9946 - binary_crossentropy:  0.8126 - auc:  0.8097 - val_binary_crossentropy:  0.6400 - val_auc:  0.7332
Epoch 6/20
40s - loss:  0.9035 - binary_crossentropy:  0.7337 - auc:  0.8332 - val_binary_crossentropy:  0.9145 - val_auc:  0.7172
Epoch 7/20
40s - loss:  0.7684 - binary_crossentropy:  0.6286 - auc:  0.8517 - val_binary_crossentropy:  0.6146 - val_auc:  0.7276
Epo

In [ ]:
from sklearn.metrics import log_loss, roc_auc_score,accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score,roc_curve
import statsmodels.api as sm
from sklearn.metrics import precision_score,recall_score
pred_ans = model.predict(test_model_input, 256)
ypred=[]
for i in pred_ans:
    ypred.append(round(i[0]))
print("")
print("test LogLoss", round(log_loss(test[target].values, pred_ans), 4))
print("test roc_auc_score", round(roc_auc_score(test[target].values, pred_ans), 4))
print("test f1 score", round(f1_score(test[target].values,ypred), 4))
print("test precision score", round(precision_score(test[target].values,ypred), 4))
print("test recall score", round(recall_score(test[target].values,ypred), 4))
print("test accuracy score", round(accuracy_score(test[target].values,ypred), 4))


test LogLoss 1.1615
test roc_auc_score 0.7027
test f1 score 0.3924
test precision score 0.5566
test recall score 0.303
test accuracy score 0.7611


## Wide & Deep

In [ ]:
from deepctr_torch.models import WDL
model =  WDL(linear_feature_columns=linear_feature_columns, dnn_feature_columns=dnn_feature_columns,
                   task='binary',dnn_hidden_units=(512,128, 512,256),  l2_reg_linear=0.1,
                   l2_reg_embedding=1e-1,  dnn_use_bn=True,device=device,dnn_activation='sigmoid')
model.compile("adagrad", "binary_crossentropy",
                  metrics=["binary_crossentropy", "auc"], )
history = model.fit(train_model_input, train[target].values, batch_size=2024, epochs=20, verbose=2,
                        validation_split=0.15)

cuda:0
Train on 680000 samples, validate on 120000 samples, 336 steps per epoch
Epoch 1/20
16s - loss:  3.5523 - binary_crossentropy:  2.2904 - auc:  0.6547 - val_binary_crossentropy:  0.8892 - val_auc:  0.7127
Epoch 2/20
16s - loss:  2.2541 - binary_crossentropy:  1.6838 - auc:  0.6667 - val_binary_crossentropy:  1.2030 - val_auc:  0.6830
Epoch 3/20
16s - loss:  1.5384 - binary_crossentropy:  1.2505 - auc:  0.6878 - val_binary_crossentropy:  1.6311 - val_auc:  0.6744
Epoch 4/20
16s - loss:  1.4286 - binary_crossentropy:  1.1845 - auc:  0.6955 - val_binary_crossentropy:  1.4779 - val_auc:  0.6783
Epoch 5/20
16s - loss:  1.1681 - binary_crossentropy:  0.9994 - auc:  0.7074 - val_binary_crossentropy:  0.5632 - val_auc:  0.7360
Epoch 6/20
16s - loss:  1.1163 - binary_crossentropy:  0.9570 - auc:  0.7129 - val_binary_crossentropy:  1.3636 - val_auc:  0.6856
Epoch 7/20
16s - loss:  1.0214 - binary_crossentropy:  0.8913 - auc:  0.7179 - val_binary_crossentropy:  0.5231 - val_auc:  0.7417
Epo

In [ ]:
from sklearn.metrics import log_loss, roc_auc_score,accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score,roc_curve
import statsmodels.api as sm
from sklearn.metrics import precision_score,recall_score
pred_ans = model.predict(test_model_input, 256)
ypred=[]
for i in pred_ans:
    ypred.append(round(i[0]))
print("")
print("test LogLoss", round(log_loss(test[target].values, pred_ans), 4))
print("test roc_auc_score", round(roc_auc_score(test[target].values, pred_ans), 4))
print("test f1 score", round(f1_score(test[target].values,ypred), 4))
print("test precision score", round(precision_score(test[target].values,ypred), 4))
print("test recall score", round(recall_score(test[target].values,ypred), 4))
print("test accuracy score", round(accuracy_score(test[target].values,ypred), 4))


test LogLoss 1.0037
test roc_auc_score 0.706
test f1 score 0.2593
test precision score 0.7082
test recall score 0.1587
test accuracy score 0.7692


##  Deep Cross

In [ ]:
from deepctr_torch.models import DCN
model =  DCN(linear_feature_columns=linear_feature_columns, dnn_feature_columns=dnn_feature_columns,
                   task='binary',dnn_hidden_units=(256,128),  l2_reg_linear=0.0001,
                   l2_reg_embedding=1e-4,  dnn_use_bn=True,device=device,dnn_activation='sigmoid')
model.compile("adam", "binary_crossentropy",
                  metrics=["binary_crossentropy", "auc"], )
history = model.fit(train_model_input, train[target].values, batch_size=2024, epochs=20, verbose=2,
                        validation_split=0.15)

cuda:0
Train on 680000 samples, validate on 120000 samples, 336 steps per epoch
Epoch 1/20
17s - loss:  54.3809 - binary_crossentropy:  19.7706 - auc:  0.4801 - val_binary_crossentropy:  8.7518 - val_auc:  0.5752
Epoch 2/20
17s - loss:  21.3672 - binary_crossentropy:  7.8518 - auc:  0.5753 - val_binary_crossentropy:  7.3706 - val_auc:  0.5692
Epoch 3/20
17s - loss:  19.7732 - binary_crossentropy:  7.2657 - auc:  0.5779 - val_binary_crossentropy:  7.2622 - val_auc:  0.5719
Epoch 4/20
17s - loss:  26.6021 - binary_crossentropy:  9.7596 - auc:  0.5579 - val_binary_crossentropy:  7.1284 - val_auc:  0.5675
Epoch 5/20
16s - loss:  19.1355 - binary_crossentropy:  7.0037 - auc:  0.5760 - val_binary_crossentropy:  7.3664 - val_auc:  0.5709
Epoch 6/20
17s - loss:  33.5523 - binary_crossentropy:  12.2286 - auc:  0.5444 - val_binary_crossentropy:  23.8103 - val_auc:  0.4501
Epoch 7/20
16s - loss:  65.3158 - binary_crossentropy:  23.6491 - auc:  0.4487 - val_binary_crossentropy:  23.4612 - val_auc:

In [ ]:
from sklearn.metrics import log_loss, roc_auc_score,accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score,roc_curve
import statsmodels.api as sm
from sklearn.metrics import precision_score,recall_score
pred_ans = model.predict(test_model_input, 256)
ypred=[]
for i in pred_ans:
    ypred.append(round(i[0]))
print("")
print("test LogLoss", round(log_loss(test[target].values, pred_ans), 4))
print("test roc_auc_score", round(roc_auc_score(test[target].values, pred_ans), 4))
print("test f1 score", round(f1_score(test[target].values,ypred), 4))
print("test precision score", round(precision_score(test[target].values,ypred), 4))
print("test recall score", round(recall_score(test[target].values,ypred), 4))
print("test accuracy score", round(accuracy_score(test[target].values,ypred), 4))


test LogLoss 7.5099
test roc_auc_score 0.5741
test f1 score 0.268
test precision score 0.5295
test recall score 0.1794
test accuracy score 0.7505
